In [ ]:
"""
Purpose: To Create a table that will hold all of the IDs that will be 
decimated because came from the nuclei


"""

# Getting the unique segment_ids

In [ ]:
import numpy as np

In [ ]:
new_seg_ids_files = np.load("../Mesh_downloading/unique_root_ids.npz",allow_pickle=True)
new_seg_ids = new_seg_ids_files["segment_ids"]
new_seg_ids.shape

In [ ]:
new_seg_ids

# Linking to the Database

In [1]:
from os import sys
sys.path.append("/meshAfterParty/")
from importlib import reload

import datajoint as dj
from pathlib import Path

import datajoint_utils as du
du = reload(du)

WARNING - 2020-11-14 07:35:45,638 - trimesh_repair - Need to pip install annotationframeworkclient to repair mesh with pychunkedgraph
WARNING - 2020-11-14 07:35:45,640 - trimesh_io - Need to pip install annotationframeworkclient to use dataset_name parameters
INFO - 2020-11-14 07:35:46,001 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-14 07:35:46,002 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-14 07:35:46,031 - connection - C

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-14 07:35:46,336 - settings - Setting enable_python_native_blobs to True


In [2]:
import minfig
du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-14 07:35:46,351 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-14 07:35:46,352 - settings - Setting database.user to celiib
INFO - 2020-11-14 07:35:46,353 - settings - Setting database.password to newceliipass
INFO - 2020-11-14 07:35:46,355 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-14 07:35:46,527 - settings - Setting enable_python_native_blobs to True


Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


# The table that will store it

In [ ]:
minnie.Mesh()

In [ ]:
@schema
class SegToDecimateFromNuclei(dj.Manual):
    definition="""
    segment_id : bigint unsigned  #segment id for those to be decimated
    """

In [ ]:
dict_of_seg = [dict(segment_id=k) for k in new_seg_ids]
len(dict_of_seg)

In [ ]:
minnie.SegToDecimateFromNuclei.insert(dict_of_seg,skip_duplicates=True)

In [ ]:
minnie.SegToDecimateFromNuclei()

# Investigating How many of the meshes we can decimate

In [ ]:
minnie.Mesh() & minnie.SegToDecimateFromNuclei()

In [ ]:
89592/114784

In [13]:
len(minnie.Decimation.proj() & minnie.SegToDecimateFromNuclei())

20298

In [ ]:
20295

In [16]:
(schema.jobs & "table_name='__decimation'" & "timestamp > '2020-11-13 00:00:00'").delete()